In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 90)

### Загрузка данных 

In [3]:
df = pd.read_csv("owid-covid-data.csv")
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.125,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.125,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,40099462.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204764,ZWE,Africa,Zimbabwe,2022-07-24,256254.0,8.0,24.429,5572.0,0.0,0.857,16022.360,0.500,1.527,348.391,0.000,0.054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15993524.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
204765,ZWE,Africa,Zimbabwe,2022-07-25,256270.0,16.0,26.714,5572.0,0.0,0.857,16023.360,1.000,1.670,348.391,0.000,0.054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15993524.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


In [4]:
df = pd.DataFrame(df[["continent","location","population","date","total_cases", "new_cases","total_deaths","new_deaths","total_vaccinations",
                      "new_vaccinations","stringency_index","median_age","aged_65_older","aged_70_older","gdp_per_capita",
                      "extreme_poverty","cardiovasc_death_rate","diabetes_prevalence","female_smokers","male_smokers"]])

### Предварительный анализ с использованием базы данных SQL

#### Создание и заполнение базы данных

In [5]:
conn = sqlite3 .connect(":memory:") 
cursor = conn.cursor()

cursor.execute("""DROP TABLE IF EXISTS test;""")
cursor.execute("""CREATE TABLE covid_info(
                
                continent VARCHAR(40),
                location VARCHAR(40),
                population FLOAT,
                date DATE,
                total_cases FLOAT, 
                new_cases FLOAT,
                total_deaths FLOAT,
                new_deaths FLOAT,
                total_vaccinations FLOAT,
                new_vaccinations FLOAT,
                stringency_index FLOAT,
                median_age FLOAT,
                aged_65_older FLOAT,
                aged_70_older FLOAT,
                gdp_per_capita FLOAT,
                extreme_poverty FLOAT,
                cardiovasc_death_rate FLOAT,
                diabetes_prevalence FLOAT,
                female_smokers FLOAT,
                male_smokers FLOAT
);""")


In [6]:
data = df.to_numpy()
data = data.tolist()

cursor.executemany("""INSERT INTO covid_info VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", data)

In [7]:
cursor.execute("""
Select *
From covid_info
Where continent is not null 
order by 4
""")

pd.DataFrame(cursor.fetchall(), columns = df.columns.to_numpy())

,continent,location,population,date,total_cases,new_cases,total_deaths,new_deaths,total_vaccinations,new_vaccinations,stringency_index,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers
0,South America,Argentina,45276780.0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0,31.9,11.198,7.441,18933.907,0.6,191.032,5.50,16.2,27.7
1,North America,Mexico,126705138.0,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.3,6.857,4.321,17336.469,2.5,152.783,13.06,6.9,21.4
2,South America,Argentina,45276780.0,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,0.0,31.9,11.198,7.441,18933.907,0.6,191.032,5.50,16.2,27.7
3,North America,Mexico,126705138.0,2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.3,6.857,4.321,17336.469,2.5,152.783,13.06,6.9,21.4
4,South America,Argentina,45276780.0,2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0,31.9,11.198,7.441,18933.907,0.6,191.032,5.50,16.2,27.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192923,Asia,Vietnam,97468028.0,2022-07-28,10774679.0,1699.0,43092.0,0.0,NaN,NaN,NaN,32.6,7.150,4.718,6171.884,2.0,245.465,6.00,1.0,45.9
192924,Oceania,Wallis and Futuna,11627.0,2022-07-28,761.0,NaN,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192925,Asia,Yemen,32981641.0,2022-07-28,11849.0,0.0,2149.0,0.0,NaN,NaN,NaN,20.3,2.922,1.583,1479.147,18.8,495.003,5.35,7.6,29.2
192926,Africa,Zambia,19473125.0,2022-07-28,329483.0,NaN,4015.0,0.0,NaN,NaN,NaN,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7


#### Запрос показывающий процент заболевших

In [8]:
cursor.execute("""
Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From covid_info
Where location is "Russia"
order by 2 
;""")

data = pd.DataFrame(cursor.fetchall(), columns = ["Страна", "Дата", "Население", "Заболевшие","Процент заболевших"])
data.fillna(0,inplace = True)
data.sample(10)

,Страна,Дата,Население,Заболевшие,Процент заболевших
476,Russia,2021-05-21,145102755.0,4926717.0,3.395330
646,Russia,2021-11-07,145102755.0,8651561.0,5.962369
664,Russia,2021-11-25,145102755.0,9303751.0,6.411836
461,Russia,2021-05-06,145102755.0,4799872.0,3.307912
591,Russia,2021-09-13,145102755.0,7055296.0,4.862276
211,Russia,2020-08-29,145102755.0,982573.0,0.677157
606,Russia,2021-09-28,145102755.0,7355883.0,5.069430
64,Russia,2020-04-04,145102755.0,4731.0,0.003260
132,Russia,2020-06-11,145102755.0,501800.0,0.345824
512,Russia,2021-06-26,145102755.0,5367317.0,3.698977


#### Запрос показывающий процент смертности 

In [9]:
cursor.execute("""
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From covid_info
Where location is "Russia"
and continent is not null 
order by 2
""")

data = pd.DataFrame(cursor.fetchall(), columns = ["Страна", "Дата", "Заболевшие","Умершие", "Процент умерших"])
data.fillna(0,inplace = True)
data.sample(10)

,Страна,Дата,Заболевшие,Умершие,Процент умерших
641,Russia,2021-11-02,8455232.0,236462.0,2.796635
851,Russia,2022-05-31,18063880.0,371464.0,2.056391
498,Russia,2021-06-12,5133938.0,123961.0,2.414540
149,Russia,2020-06-28,633563.0,9060.0,1.430008
155,Russia,2020-07-04,673564.0,10011.0,1.486273
895,Russia,2022-07-14,18206982.0,374024.0,2.054289
555,Russia,2021-08-08,6362641.0,162109.0,2.547826
220,Russia,2020-09-07,1027334.0,17818.0,1.734392
138,Russia,2020-06-17,552549.0,7468.0,1.351554
417,Russia,2021-03-23,4424595.0,94231.0,2.129709


#### Запрос показывающий страны с высоким уровнем заражения

In [10]:
cursor.execute("""
Select Location, Population, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected
From covid_info
Group by Location, Population
order by PercentPopulationInfected desc
;""")

data = pd.DataFrame(cursor.fetchall(), columns = ["Страна", "Население", "Макс.число заболевших", "Процент заболевших"])
data.fillna(0,inplace = True)
data

,Страна,Население,Макс.число заболевших,Процент заболевших
0,Faeroe Islands,52888.0,34658.0,65.530933
1,Cyprus,896007.0,556048.0,62.058444
2,Gibraltar,32670.0,19902.0,60.918274
3,San Marino,33746.0,19752.0,58.531381
4,Andorra,79034.0,45508.0,57.580282
...,...,...,...,...
239,Tokelau,1849.0,0.0,0.000000
240,Turkmenistan,6341855.0,0.0,0.000000
241,Tuvalu,11204.0,0.0,0.000000
242,United States Virgin Islands,100091.0,0.0,0.000000


#### Запрос показывающий страны с высоким уровнем смертности

In [11]:
cursor.execute("""
Select Location, Population, MAX(total_deaths) as HighestDeathsCount,  Max((total_deaths/population))*100 as PercentPopulationDeaths
From covid_info
Where continent is not null 
Group by Location
order by PercentPopulationDeaths desc
;""")


data = pd.DataFrame(cursor.fetchall(), columns = ["Страна", "Население", "Макс.число умерших", "Процент умерших"])
data.fillna(0,inplace = True)
data

,Страна,Население,Макс.число умерших,Процент умерших
0,Peru,33715472.0,214120.0,0.635079
1,Bulgaria,6885868.0,37360.0,0.542561
2,Bosnia and Herzegovina,3270943.0,15851.0,0.484600
3,Hungary,9709786.0,46790.0,0.481885
4,Georgia,3757980.0,16859.0,0.448619
...,...,...,...,...
226,Marshall Islands,42050.0,0.0,0.000000
227,Jersey,109618.0,0.0,0.000000
228,Guernsey,63065.0,0.0,0.000000
229,Guam,170534.0,0.0,0.000000
